<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/collections_demos/bonemarrowwsi_pediatricleukemia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TBD: better title - BoneMarrowWSI-PediatricLeukemia


## Background

The analysis of bone marrow aspirate smears serves as the foundation for assessing haematopoiesis (the process of blood cell formation in the bone marrow) and amongst other systemic diseses, the diagnosis of leukemia. This notebook introduces the `BoneMarrowWSI-PediatricLeukemia` collection, a commprehensive dataset of bone marrow smears and cell annotations. The dataset is presented in detail in [this preprint](https://www.arxiv.org/pdf/2509.15895) and is publicly available via the [Imaging Data Commons](https://portal.imaging.datacommons.cancer.gov/). 

- **Images**: The `BoneMarrowWSI-PediatricLeukemia` dataset comprises bone marrow aspirate smear WSIs for almost 250 pediatric cases of leukemia, including acute lymphoid leukemia (ALL), acute myeloid leukemia (AML), and chronic myeloid leukemia (CML). The smears were prepared for the initial diagnosis (i.e., without prior treatment), stained in accordance with the Pappenheim method, and scanned at 40x magnification.
- **Annotations**: The images have been annotated with rectangular regions of interest (ROI) of the evaluable monolayer area and more than 40000 cell bounding box annotations have been placed (with few exceptions) within the ROIs. For a subset of them all cells and other haematological structures have additionally been labelled by multiple experts in a consensus labeling approach with 49 distinct (cell type) classes. The consensus labelling approach worked as follows: each bounding box was successively labelled by different experts in so-called "annotation sessions" until (a) the bounding box had been labelled by at least two experts, and (b) the most frequent label had constituted at least half of all labels given to that bounding box (and had then been termed "consensus class"). In summary, the following annotations are available:  

    - For each slide: ROI annotations of the monolayer area for each slide
    - For some slides: Unlabeled cell bounding boxes
    - For some slides: Cell bounding boxes with cell type labels for each annotation session plus the finally obtained consensus.

Both images and annotations are made available in the DICOM format. For the annotations, more specifically, the **DICOM Microscopy Bulk Simple Annotation format (ANNs)** was used, which is one of multiple options to store pathology annotation data in DICOM. As a general introduction to this format, we recommend having a look at [this tutorial notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/pathomics/microscopy_dicom_ann_intro.ipynb).
This notebook demonstrates **how to access and explore** the `BoneMarrowWSI-PediatricLeukemia` collection with a focus on its extensive annotation data. 


<img src="https://raw.githubusercontent.com/ImagingDataCommons/IDC-Tutorials/master/notebooks/pathomics/bmdeep_annotations_example.png" alt="Example visualization of BoneMarrowWSI-PediatricLeukemia annotations" width="1000"/>



## Prerequisites
**Installations**
* **Install highdicom:** [highdicom](https://highdicom.readthedocs.io/en/latest/introduction.html) was specifically designed to work with DICOM objects holding image-derived information, e.g. annotations and measurements. Detailed information on highdicom's functionality can be found in its [user guide](https://highdicom.readthedocs.io/en/latest/usage.html).
* **Install wsidicom:** The [wsidicom](https://pypi.org/project/wsidicom/) Python package provides functionality to open and extract image or metadata from WSIs.
* **Install idc-index:** The Python package [idc-index](https://pypi.org/project/idc-index/) facilitates queries of the basic metadata and download of DICOM files hosted by the IDC.

In [1]:
%%capture
!pip install highdicom
!pip install wsidicom
!pip install idc-index --upgrade

## Imports

In [2]:
import os
import highdicom as hd
from idc_index import IDCClient
import pandas as pd
from google.cloud import storage
from pathlib import Path
from typing import List, Union

## Finding the `BoneMarrowWSI-PediatricLeukemia` dataset on IDC
To access and download image and ANNs files, we utilize the Python package [idc-index](https://github.com/ImagingDataCommons/idc-index) and fetch `ann_index` and `ann_group_index`, specific to DICOM ANN objects. You can have a look [here](https://idc-index.readthedocs.io/en/latest/indices_reference.html) for details on all available indices in idc-index.

In [3]:
idc_client = IDCClient() # set-up idc_client
idc_client.fetch_index('ann_index') # fetch index for ANN objects
idc_client.fetch_index('ann_group_index') # fetch index for ANN groups

First, we count the number of slides (=distinct SeriesInstanceUIDs of SM modality) available in the `BoneMarrowWSI-PediatricLeukemia` collection:

In [4]:
query_slide_count = '''
SELECT COUNT(DISTINCT SeriesInstanceUID)
FROM
    index
WHERE
    collection_id = 'bonemarrowwsi_pediatricleukemia' AND Modality='SM'
'''
print(idc_client.sql_query(query_slide_count))

   count(DISTINCT SeriesInstanceUID)
0                                246


Next, we have a look at the available annotation (ANN) files. The following query collects information about ANN files on series-level from idc-index's `ann_index`.

# TOdO: explain why join on SeriesInstanceUID

In [5]:
query_anns = '''
SELECT
    ann_index.SeriesInstanceUID,
    ann_index.referenced_SeriesInstanceUID,
    index.SeriesDescription,
    index.StudyInstanceUID,
FROM
    ann_index
JOIN index
  ON index.SeriesInstanceUID = ann_index.SeriesInstanceUID
WHERE
    index.collection_id = 'bonemarrowwsi_pediatricleukemia'
ORDER BY
    ann_index.referenced_SeriesInstanceUID
'''
annotations = idc_client.sql_query(query_anns)
display(annotations)

,SeriesInstanceUID,referenced_SeriesInstanceUID,SeriesDescription,StudyInstanceUID
0,1.2.826.0.1.3680043.10.511.3.91776015271042872...,1.2.826.0.1.3680043.8.498.11363257719279170754...,Session 1: Cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.32796004339808195676...
1,1.2.826.0.1.3680043.10.511.3.94539084193943051...,1.2.826.0.1.3680043.8.498.11363257719279170754...,Consensus: cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.32796004339808195676...
2,1.2.826.0.1.3680043.10.511.3.70465245248160893...,1.2.826.0.1.3680043.8.498.11363257719279170754...,Session 0: Cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.32796004339808195676...
3,1.2.826.0.1.3680043.10.511.3.81099351624121702...,1.2.826.0.1.3680043.8.498.11363257719279170754...,Monolayer regions of interest for cell classif...,1.2.826.0.1.3680043.8.498.32796004339808195676...
4,1.2.826.0.1.3680043.10.511.3.46754255566554977...,1.2.826.0.1.3680043.8.498.11363257719279170754...,Session 4: Cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.32796004339808195676...
...,...,...,...,...
1028,1.2.826.0.1.3680043.10.511.3.53100158250225700...,1.2.826.0.1.3680043.8.498.99096006395418595522...,Session 4: Cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.12485116558934689418...
1029,1.2.826.0.1.3680043.10.511.3.27247460680841398...,1.2.826.0.1.3680043.8.498.99096006395418595522...,Consensus: cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.12485116558934689418...
1030,1.2.826.0.1.3680043.10.511.3.60357016090560296...,1.2.826.0.1.3680043.8.498.99096006395418595522...,Session 0: Cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.12485116558934689418...
1031,1.2.826.0.1.3680043.10.511.3.95416517312103214...,1.2.826.0.1.3680043.8.498.99096006395418595522...,Session 1: Cell bounding boxes with cell type ...,1.2.826.0.1.3680043.8.498.12485116558934689418...


We can see, that for each slide (i.e. **referencedSeriesInstanceUID**) there are multiple ANN Series. Looking at the **SeriesDescription**, we can assert what is described in the [Background](#Background) section of this notebook.

*   Each slide has "Monolayer regions of interest for cell classification" annotations.
*   For some slides, there is one ANN Series with "Unlabeled cell bounding boxes", while for others, there are multiple ANN Series containing "Cell bounding boxes with cell type labels" for different annotation sessions and the consensus labels.

ANN files contain one or multiple "AnnotationGroups" (for more explanation see the aforementioned [tutorial notebook]((https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/pathomics/microscopy_dicom_ann_intro.ipynb)) on DICOM ANNs).
Using `ann_group_index` we can get a little more information on what the individual annotation groups contain exactly. The following code cell queries the AnnotationGroupLabel - a human-readable label for the annotation group - as well as coded values from controlled vocabularies like SNOMED-CT for the category and type of the annotation.


In [21]:
query_ann_groups = '''
SELECT
    ag.AnnotationGroupLabel, -- human-readable label of annotation group
    ag.AnnotationPropertyCategory_CodeMeaning, -- Code meaning of coded value for the category of the annotation group
    ag.AnnotationPropertyType_CodeMeaning, -- Code meaning of coded value for the type of the annotation group
    ARRAY_AGG(DISTINCT i.SeriesDescription) AS series_descriptions
FROM ann_group_index ag
JOIN ann_index ai
    ON ag.SeriesInstanceUID = ai.SeriesInstanceUID
JOIN index i
    ON ag.SeriesInstanceUID = i.SeriesInstanceUID
WHERE
    i.collection_id = 'bonemarrowwsi_pediatricleukemia'
GROUP BY
    ag.AnnotationGroupLabel,
    ag.AnnotationPropertyCategory_CodeMeaning,
    ag.AnnotationPropertyType_CodeMeaning,
ORDER BY
    LEN(series_descriptions)
'''

ann_groups = idc_client.sql_query(query_ann_groups)
display(ann_groups)

,AnnotationGroupLabel,AnnotationPropertyCategory_CodeMeaning,AnnotationPropertyType_CodeMeaning,series_descriptions
0,region_of_interest,Spatial and relational concepts,Selected region,[Monolayer regions of interest for cell classi...
1,haematological_structure,Anatomical structure,Structure of haematological system,[Unlabeled cell bounding boxes]
2,megakaryocyte,Anatomical structure,Megakaryocyte,[Session 1: Cell bounding boxes with cell type...
3,promegakaryocyte,Anatomical structure,Promegakaryocyte,[Consensus: cell bounding boxes with cell type...
4,micromegakaryocyte,Anatomical structure,Micromegakaryocyte,[Session 1: Cell bounding boxes with cell type...
5,lymphoid_precursor_cell,Anatomical structure,Lymphoid precursor cell,[Session 1: Cell bounding boxes with cell type...
6,basophilic_metamyelocyte,Anatomical structure,Basophilic metamyelocyte,[Session 1: Cell bounding boxes with cell type...
7,pseudo_gaucher_cell,Anatomical structure,Gaucher-like cell,[Session 2: Cell bounding boxes with cell type...
8,plasma_cell,Anatomical structure,Plasma cell,[Session 2: Cell bounding boxes with cell type...
9,phagocytosis,Anatomical structure,Phagocytosis,[Session 0: Cell bounding boxes with cell type...


Relating the annotated objects to the SeriesDescription, we can see, that

- ANN Series "Monolayer regions of interest for cell classification" contain only annotations with AnnotatedPropertyType "Selected region" and the human-readable label "region_of_interest".
- "Unlabeled cell bounding boxes" contain only annotations with property type "Structure of haematological system" and the human-readable label "haematological_structure"
- all other ANN Series contain various different AnnotatedPropertyTypes and human-readable labels.

We will use this information below to filter for the monolayer regions of interest and labeled vs. unlabeled cell annotations.

## Viewing annotations


Annotations can be viewed and explored in detail on its respective slide using the Slim viewer. In the Slim viewer's interface at the bottom of the right sidebar you may select the ANN Series of interest to you from the drop-down menue, then click on `Annotation Groups` and switch the slider(s) to make annotations visible.

In [6]:
viewer_url = idc_client.get_viewer_URL(studyInstanceUID=annotations['StudyInstanceUID'][0], viewer_selector='slim')
from IPython.display import IFrame
IFrame(viewer_url, width=1500, height=900)

## Accessing annotations

### Download complete annotation collection for local access
Since the complete set of annotation series is of reasonable size it could be downloaded completely using `idc_index` as shown below and then accessed from the local disk using `highdicom`.

In [7]:
dcm_ann_dir = Path('/content/dicom_ann_annotations')
os.makedirs(dcm_ann_dir, exist_ok=True)

idc_client.download_from_selection(downloadDir=dcm_ann_dir,
                                   seriesInstanceUID=annotations['SeriesInstanceUID'].tolist(), dirTemplate=None)

For guidance on how to read the downloaded annotation files see section "Reading DICOM ANNs" of [this tutorial notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/pathomics/microscopy_dicom_ann_intro.ipynb).

### Access annotations directly from the Cloud

A more desirable approach especially for larger size datasets is to directly extract the relevant information from the objects in the cloud. The following functions `get_roi_annotations()` and `get_cell_annotations()` can be used for this approach. They extract and summarize ROIs respectively cell annotations in an easy to use pandas DataFrame.
Note, that the selection of the respective annotation files, i.e. files containing ROI annotations, labeled or unlabeled cell annotations, is done by filtering for the respective AnnotationPropertyTypes as determined above.

The following two code cells define and use `get_roi_annotations()` to select all DICOM ANNs in the `BoneMarrowWSI-PediatricLeukemia` collection that contain ROI annotations of the monolayer area.
The resulting pandas DataFrame contains
- **'SeriesInstanceUID'**: SeriesInstanceUID of the DICOM ANN Series containing the ROI annotation.
- **'roi_id'**: the ID of the ROI
- **'roi_human_label'**: its human-readable label  
- **'roi_label_code_scheme'**: Tuple of code of the ROI label and designator of the coding scheme used, e.g. (111099, DCM) which is code 111099 from DICOM coding scheme
- **'roi_label'**: Code meaning of the ROI label
- **'roi_coordinates'**: the 2D coordinates in the image coordinate system of the referenced slide level
- **'reference_SeriesInstanceUID'** and **'reference_SOPInstanceUID'**: the SeriesInstanceUID and SOPInstanceUID of the slide level the annotations refer to. reference_SeriesInstanceUID can either be obtained from ann_index or read from the ANN file directly - for consistency with reference_SOPInstanceUID the later approach was chosen here.


In [24]:
def get_roi_annotations(ann_to_process: int = 10):
    # Select all annotation groups from ANN series in the bonemarrowwsi_pediatricleukemia collection
    # whose AnnotationPropertyType is "Selected region" and AnnotationGroupLabel is "region_of_interest"
    query_roi_anns = f'''
    SELECT
        ag.SeriesInstanceUID
    FROM
        ann_group_index ag
    JOIN ann_index ai
        ON ag.SeriesInstanceUID = ai.SeriesInstanceUID
    JOIN index i
        ON ag.SeriesInstanceUID = i.SeriesInstanceUID
    WHERE
        i.collection_id = 'bonemarrowwsi_pediatricleukemia'
        AND ag.AnnotationPropertyType_CodeMeaning = 'Selected region'
        AND ag.AnnotationGroupLabel = 'region_of_interest'
    ORDER BY
        ag.SeriesInstanceUID
    LIMIT {ann_to_process}
    '''
    roi_series = idc_client.sql_query(query_roi_anns)['SeriesInstanceUID']
    rois = extract_rois(roi_series)
    return rois


def extract_rois(series_uids: List[str]) -> pd.DataFrame:
    gcs_client = storage.Client.create_anonymous_client()
    rows = []
    for series_uid in series_uids:
        file_urls = idc_client.get_series_file_URLs(seriesInstanceUID=series_uid, source_bucket_location='gcs')
        for file_url in file_urls:
            (_,_, bucket_name, folder_name, file_name) = file_url.split('/')
            bucket = gcs_client.bucket(bucket_name)
            blob = bucket.blob(f'{folder_name}/{file_name}')

            with blob.open('rb') as file_obj:
                ann = hd.ann.annread(file_obj)
                for ann_group in ann.get_annotation_groups():
                    coords = ann_group.get_graphic_data(coordinate_type='2D')
                    m_names, m_values, m_units = ann_group.get_measurements()
                    for c, m in zip(coords, m_values):
                        rows.append({
                            'SeriesInstanceUID': ann.SeriesInstanceUID,
                            'roi_id': int(m[0]), # allow empty roi_id,
                            'roi_human_label': ann_group.label,
                            'roi_label_code_scheme': (ann_group.annotated_property_type.CodeValue, ann_group.annotated_property_type.CodingSchemeDesignator),
                            'roi_label': ann_group.annotated_property_type.CodeMeaning,
                            'roi_coordinates': c,
                            'reference_SeriesInstanceUID': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                            'reference_SOPInstanceUID': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID,
                        })
    rois = pd.DataFrame(rows)
    return rois

In [25]:
# This code will run longer as you increase the number of ANN files to be processed
rois = get_roi_annotations(ann_to_process=10)
display(rois)

,SeriesInstanceUID,roi_id,roi_human_label,roi_label_code_scheme,roi_label,roi_coordinates,reference_SeriesInstanceUID,reference_SOPInstanceUID
0,1.2.826.0.1.3680043.10.511.3.10076145498370342...,1152,region_of_interest,"(111099, DCM)",Selected region,"[[97220.0, 195768.0], [99268.0, 195768.0], [99...",1.2.826.0.1.3680043.8.498.32484296459223334560...,1.2.826.0.1.3680043.8.498.87476043951163326277...
1,1.2.826.0.1.3680043.10.511.3.10076145498370342...,1153,region_of_interest,"(111099, DCM)",Selected region,"[[98892.0, 160318.0], [100940.0, 160318.0], [1...",1.2.826.0.1.3680043.8.498.32484296459223334560...,1.2.826.0.1.3680043.8.498.87476043951163326277...
2,1.2.826.0.1.3680043.10.511.3.10370508621567005...,1018,region_of_interest,"(111099, DCM)",Selected region,"[[113423.0, 326838.0], [115471.0, 326838.0], [...",1.2.826.0.1.3680043.8.498.76034665511943175311...,1.2.826.0.1.3680043.8.498.10662747416486796732...
3,1.2.826.0.1.3680043.10.511.3.10370508621567005...,1019,region_of_interest,"(111099, DCM)",Selected region,"[[30933.0, 380620.0], [32981.0, 380620.0], [32...",1.2.826.0.1.3680043.8.498.76034665511943175311...,1.2.826.0.1.3680043.8.498.10662747416486796732...
4,1.2.826.0.1.3680043.10.511.3.10509566419026275...,2037,region_of_interest,"(111099, DCM)",Selected region,"[[48237.0, 89186.0], [50285.0, 89186.0], [5028...",1.2.826.0.1.3680043.8.498.17135957873481360405...,1.2.826.0.1.3680043.8.498.61055026268823152919...
5,1.2.826.0.1.3680043.10.511.3.10509566419026275...,2038,region_of_interest,"(111099, DCM)",Selected region,"[[25890.0, 97475.0], [27938.0, 97475.0], [2793...",1.2.826.0.1.3680043.8.498.17135957873481360405...,1.2.826.0.1.3680043.8.498.61055026268823152919...
6,1.2.826.0.1.3680043.10.511.3.11224067190602751...,322,region_of_interest,"(111099, DCM)",Selected region,"[[40807.0, 31720.0], [42855.0, 31720.0], [4285...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...
7,1.2.826.0.1.3680043.10.511.3.11224067190602751...,323,region_of_interest,"(111099, DCM)",Selected region,"[[41251.0, 26908.0], [43299.0, 26908.0], [4329...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...
8,1.2.826.0.1.3680043.10.511.3.11224067190602751...,2017,region_of_interest,"(111099, DCM)",Selected region,"[[25157.0, 41317.0], [30531.0, 41317.0], [3053...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...
9,1.2.826.0.1.3680043.10.511.3.11224067190602751...,2018,region_of_interest,"(111099, DCM)",Selected region,"[[83712.0, 75137.0], [88720.0, 75137.0], [8872...",1.2.826.0.1.3680043.8.498.82223767803353692585...,1.2.826.0.1.3680043.8.498.72082594196695068782...


The following code cells define and use `get_cell_annotations()` to select all DICOM ANNs in the `BoneMarrowWSI-PediatricLeukemia` collection that contain cell annotations. By setting the parameter 'subset' to either 'labeled', 'unlabeled' or 'both', it's possible to extract either only labeled, unlabeled or all cell annotations.
The resulting pandas DataFrame contains
- **'SeriesInstanceUID'**: SeriesInstanceUID of the DICOM ANN Series containing the cell annotation.
- **'annotation_session'**: 'n/a' for the unlabeled cells, otherwise the number of the annotation session or 'consensus' for the final consensus.
- **'cell_id'**: the ID of the cell
- **'roi_id'**: if applicable, the ID of the monolayer ROI, that the cell is located within
- **'cell_human_label'**: Human-readable label of the cell
- **'cell_label_code_scheme'**: Tuple of code of the cell label and designator of the coding scheme used, e.g. (414387006, SCT) which is code 414387006 from SNOMED-CT
- **'cell_label'**: Code meaning of the cell label defined in cell_label_code_scheme e.g. 'Structure of haematological system'. Have a look at the [SNOMED Browser](https://browser.ihtsdotools.org/?perspective=full&conceptId1=414387006&edition=MAIN&release=&languages=en) for this example.
- **'cell_coordinates'**: the 2D coordinates in the image coordinate system of the referenced slide level
- **'reference_SeriesInstanceUID'** and **'reference_SOPInstanceUID'**: the SeriesInstanceUID and SOPInstanceUID of the slide level the annotations refer to. reference_SeriesInstanceUID can either be obtained from ann_index or read from the ANN file directly - for consistency with reference_SOPInstanceUID the later approach was chosen here.

In [30]:
def get_cell_annotations(subset: Union[str, None] = None, ann_to_process: int = 10) -> pd.DataFrame:
    assert subset in ['labeled', 'unlabeled', None]

    if subset == 'unlabeled':
        filter_condition = f'''
            i.collection_id = 'bonemarrowwsi_pediatricleukemia'
            AND ag.AnnotationPropertyType_CodeMeaning = 'Structure of haematological system'
            AND ag.AnnotationGroupLabel = 'haematological_structure'
            '''
    elif subset == 'labeled':
        filter_condition = f'''
            i.collection_id = 'bonemarrowwsi_pediatricleukemia'
            AND NOT (ag.AnnotationPropertyType_CodeMeaning = 'Structure of haematological system'
                    AND ag.AnnotationGroupLabel = 'haematological_structure')
            '''
    else:
        filter_condition = '''i.collection_id = 'bonemarrowwsi_pediatricleukemia'''

    query_cell_anns = f'''
      SELECT
          ag.SeriesInstanceUID
      FROM
          ann_group_index ag
      JOIN ann_index AS ai
          ON ag.SeriesInstanceUID = ai.SeriesInstanceUID
      JOIN index i
          ON ag.SeriesInstanceUID = i.SeriesInstanceUID
      WHERE
          {filter_condition}
      ORDER BY
          ag.SeriesInstanceUID
      LIMIT {ann_to_process}
      '''
    cell_series = idc_client.sql_query(query_cell_anns)['SeriesInstanceUID']
    cells = extract_cells(cell_series)
    return cells


def extract_cells(series_uids: List[str]) -> pd.DataFrame:
    gcs_client = storage.Client.create_anonymous_client()
    rows = []
    for series_uid in series_uids:
        file_urls = idc_client.get_series_file_URLs(seriesInstanceUID=series_uid, source_bucket_location='gcs')
        for file_url in file_urls:
            (_,_, bucket_name, folder_name, file_name) = file_url.split('/')
            bucket = gcs_client.bucket(bucket_name)
            blob = bucket.blob(f'{folder_name}/{file_name}')

            with blob.open('rb') as file_obj:
                ann = hd.ann.annread(file_obj)
                for ann_group in ann.get_annotation_groups():
                    coords = ann_group.get_graphic_data(coordinate_type='2D')
                    m_names, m_values, m_units = ann_group.get_measurements()
                    for c, m in zip(coords, m_values):
                        rows.append({
                            'SeriesInstanceUID': ann.SeriesInstanceUID,
                            'annotation_session': get_annotation_session(ann),
                            'cell_id': int(m[0]),
                            'roi_id': int(m[1]) if m.size > 1 else None, # allow empty roi_id,
                            'cell_human_label': ann_group.label,
                            'cell_label': ann_group.annotated_property_type.meaning,
                            'cell_label_code_scheme': (ann_group.annotated_property_type.value, ann_group.annotated_property_type.scheme_designator),
                            'cell_coordinates': c,
                            'reference_SeriesInstanceUID': ann.ReferencedSeriesSequence[0].SeriesInstanceUID,
                            'reference_SOPInstanceUID': ann.ReferencedImageSequence[0].ReferencedSOPInstanceUID
                        })
    cells = pd.DataFrame(rows)
    return cells


def get_annotation_session(ann: hd.ann.sop.MicroscopyBulkSimpleAnnotations) -> str:
    if 'unlabeled' in ann.SeriesDescription.lower():
        return 'n/a'
    return ann.SeriesDescription.split(':')[0]

In [31]:
# This code will run longer as you increase the number of ANN files to be processed
unlabeled_cells = get_cell_annotations(subset='unlabeled', ann_to_process=10)
display(unlabeled_cells)

,SeriesInstanceUID,annotation_session,cell_id,roi_id,cell_human_label,cell_label,cell_label_code_scheme,cell_coordinates,reference_SeriesInstanceUID,reference_SOPInstanceUID
0,1.2.826.0.1.3680043.10.511.3.10323924147988442...,n/a,114518,1174,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[92032.0, 215473.0], [92138.0, 215473.0], [92...",1.2.826.0.1.3680043.8.498.79371179824920998262...,1.2.826.0.1.3680043.8.498.81643514503975082734...
1,1.2.826.0.1.3680043.10.511.3.10323924147988442...,n/a,114519,1174,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[91986.0, 215538.0], [92127.0, 215538.0], [92...",1.2.826.0.1.3680043.8.498.79371179824920998262...,1.2.826.0.1.3680043.8.498.81643514503975082734...
2,1.2.826.0.1.3680043.10.511.3.10323924147988442...,n/a,114520,1174,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[92103.0, 215517.0], [92199.0, 215517.0], [92...",1.2.826.0.1.3680043.8.498.79371179824920998262...,1.2.826.0.1.3680043.8.498.81643514503975082734...
3,1.2.826.0.1.3680043.10.511.3.10323924147988442...,n/a,114521,1174,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[92451.0, 215285.0], [92557.0, 215285.0], [92...",1.2.826.0.1.3680043.8.498.79371179824920998262...,1.2.826.0.1.3680043.8.498.81643514503975082734...
4,1.2.826.0.1.3680043.10.511.3.10323924147988442...,n/a,114522,1174,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[92709.0, 215442.0], [92813.0, 215442.0], [92...",1.2.826.0.1.3680043.8.498.79371179824920998262...,1.2.826.0.1.3680043.8.498.81643514503975082734...
...,...,...,...,...,...,...,...,...,...,...
2052,1.2.826.0.1.3680043.10.511.3.20449813712117001...,n/a,107523,1097,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[100110.0, 263966.0], [100302.0, 263966.0], [...",1.2.826.0.1.3680043.8.498.31088998307342714143...,1.2.826.0.1.3680043.8.498.98991581155690605814...
2053,1.2.826.0.1.3680043.10.511.3.20449813712117001...,n/a,107524,1097,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[100814.0, 263911.0], [100991.0, 263911.0], [...",1.2.826.0.1.3680043.8.498.31088998307342714143...,1.2.826.0.1.3680043.8.498.98991581155690605814...
2054,1.2.826.0.1.3680043.10.511.3.20449813712117001...,n/a,107525,1097,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[100782.0, 264035.0], [100938.0, 264035.0], [...",1.2.826.0.1.3680043.8.498.31088998307342714143...,1.2.826.0.1.3680043.8.498.98991581155690605814...
2055,1.2.826.0.1.3680043.10.511.3.20449813712117001...,n/a,107526,1097,haematological_structure,Structure of haematological system,"(414387006, SCT)","[[100683.0, 263013.0], [100858.0, 263013.0], [...",1.2.826.0.1.3680043.8.498.31088998307342714143...,1.2.826.0.1.3680043.8.498.98991581155690605814...


In [32]:
# This code will run longer as you increase the number of ANN files to be processed
labeled_cells = get_cell_annotations(subset='labeled', ann_to_process=10)
sorted_cell_labels = labeled_cells.sort_values(by=['reference_SOPInstanceUID', 'cell_id', 'annotation_session'])
display(sorted_cell_labels.style.hide(axis='index')) # don't show row index

SeriesInstanceUID,annotation_session,cell_id,roi_id,cell_human_label,cell_label,cell_label_code_scheme,cell_coordinates,reference_SeriesInstanceUID,reference_SOPInstanceUID
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31009,227,lymphocytic_blast,Lymphoblast,"('15433008', 'SCT')",[[84698. 77060.] [84825. 77060.] [84825. 77171.] [84698. 77171.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31009,227,lymphocytic_blast,Lymphoblast,"('15433008', 'SCT')",[[84698. 77060.] [84825. 77060.] [84825. 77171.] [84698. 77171.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31009,227,lymphocytic_blast,Lymphoblast,"('15433008', 'SCT')",[[84698. 77060.] [84825. 77060.] [84825. 77171.] [84698. 77171.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31009,227,lymphocytic_blast,Lymphoblast,"('15433008', 'SCT')",[[84698. 77060.] [84825. 77060.] [84825. 77171.] [84698. 77171.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31009,227,lymphocytic_blast,Lymphoblast,"('15433008', 'SCT')",[[84698. 77060.] [84825. 77060.] [84825. 77171.] [84698. 77171.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31020,227,no_consensus_found,Structure of haematological system,"('414387006', 'SCT')",[[85184. 77287.] [85337. 77287.] [85337. 77452.] [85184. 77452.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31020,227,unknown_blast,Blast cell,"('312256009', 'SCT')",[[85184. 77287.] [85337. 77287.] [85337. 77452.] [85184. 77452.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31020,227,no_consensus_found,Structure of haematological system,"('414387006', 'SCT')",[[85184. 77287.] [85337. 77287.] [85337. 77452.] [85184. 77452.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31020,227,unknown_blast,Blast cell,"('312256009', 'SCT')",[[85184. 77287.] [85337. 77287.] [85337. 77452.] [85184. 77452.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445
1.2.826.0.1.3680043.10.511.3.10000458259578156824372970041815666,Session 2,31020,227,no_consensus_found,Structure of haematological system,"('414387006', 'SCT')",[[85184. 77287.] [85337. 77287.] [85337. 77452.] [85184. 77452.]],1.2.826.0.1.3680043.8.498.34463067830784590559581625469113774687,1.2.826.0.1.3680043.8.498.17910403824742244732861652278168977445


## How to use the `BoneMarrowWSI-PediatricLeukemia` annotations
The `BoneMarrowWSI-PediatricLeukemia` collection stands out due to the extensive amount of information contained in its annotations. More than 40000 cells are annotated with bounding boxes suitable for training **cell detection models**, 28000 of those additionally received expert-generated class labels for **cell type classification** tasks. Particularly noteworthy is the uncertainty information embedded in the consensus labelling process, giving insight into which cell types are particularly challenging to determine or easy to confuse with others.
In the cell below, we catch some of those cases:

In [33]:
grouped_cell_labels = sorted_cell_labels.groupby('cell_id').agg({'cell_label': list, 'cell_label_code_scheme': list,
                                                              'reference_SOPInstanceUID': 'first',
                                                              'cell_coordinates': 'first'})
uncertain = grouped_cell_labels['cell_label'].apply(lambda x: len(set(x)) > 1)
display(grouped_cell_labels[uncertain])

,cell_label,cell_label_code_scheme,reference_SOPInstanceUID,cell_coordinates
cell_id,,,,
1519,"[Band neutrophil, Structure of haematological ...","[(702697008, SCT), (414387006, SCT), (70269700...",1.2.826.0.1.3680043.8.498.80225091671086795258...,"[[67463.0, 181988.0], [67607.0, 181988.0], [67..."
1553,"[Hypogranular white blood cell, Structure of h...","[(250292003, SCT), (414387006, SCT), (25029200...",1.2.826.0.1.3680043.8.498.80225091671086795258...,"[[68431.0, 181909.0], [68591.0, 181909.0], [68..."
1644,"[Hypogranular white blood cell, Structure of h...","[(250292003, SCT), (414387006, SCT), (25029200...",1.2.826.0.1.3680043.8.498.80225091671086795258...,"[[67903.0, 183726.0], [68035.0, 183726.0], [68..."
1714,"[Structure of haematological system, Blast cel...","[(414387006, SCT), (312256009, SCT), (41438700...",1.2.826.0.1.3680043.8.498.80225091671086795258...,"[[68699.0, 182652.0], [68812.0, 182652.0], [68..."
1835,"[Structure of haematological system, Unusable ...","[(414387006, SCT), (111235, DCM), (414387006, ...",1.2.826.0.1.3680043.8.498.80225091671086795258...,"[[86437.0, 189648.0], [86554.0, 189648.0], [86..."
1926,"[Damage, Structure of haematological system, D...","[(37782003, SCT), (414387006, SCT), (37782003,...",1.2.826.0.1.3680043.8.498.80225091671086795258...,"[[85642.0, 189293.0], [85691.0, 189293.0], [85..."
31020,"[Structure of haematological system, Blast cel...","[(414387006, SCT), (312256009, SCT), (41438700...",1.2.826.0.1.3680043.8.498.17910403824742244732...,"[[85184.0, 77287.0], [85337.0, 77287.0], [8533..."


# Next steps

Share your feedback or ask questions about this notebook in IDC Forum: https://discourse.canceridc.dev.

If you are interested in tissue type annotations or want to learn about DICOM Structured Reporting, you can take a look at [this notebook](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/collections_demos/rms_mutation_prediction/RMS-Mutation-Prediction-Expert-Annotations_exploration.ipynb) navigating expert-generated region annotations for rhabdomyosarcoma tumor slides.